# Havi rendszeres lekérdezések futtatása

In [3]:
import pyodbc
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime, timezone
import matplotlib.pyplot as plt

%matplotlib inline

## Havi változás-halmaz adatcsomag előállítása -> .csv

In [3]:
df_already_scanned = pd.read_csv("already_scanned.csv", sep=";")

In [4]:
paramList = []
entry = []
x = len(df_already_scanned)

In [5]:
entry.append('sumRecord_Number')
entry.append(df_already_scanned['sourceRecordNumber'].sum())
paramList.append(entry)
entry = []
entry.append('minSumPriceBill')
entry.append(df_already_scanned['dfSumPriceBill'].min())
paramList.append(entry)
entry = []
entry.append('maxSumPriceBill')
entry.append(df_already_scanned['dfSumPriceBill'].max())
paramList.append(entry)
entry = []
entry.append('avgSumPriceBill')
entry.append(df_already_scanned['dfSumPriceBill'].mean())
paramList.append(entry)
entry = []

In [6]:
penultSumPriceBill = df_already_scanned.loc[x-2:x-2, ['dfSumPriceBill']].iloc[0,0]
entry.append('penultSumPriceBill')
entry.append(penultSumPriceBill)
paramList.append(entry)
entry = []

lastSumPriceBill = df_already_scanned.loc[x-1:x, ['dfSumPriceBill']].iloc[0,0]
entry.append('lastSumPriceBill')
entry.append(lastSumPriceBill)
paramList.append(entry)
entry = []

changeLastSumPriceBill = ((lastSumPriceBill/penultSumPriceBill)-1)*100
entry.append('changeLastSumPriceBill')
entry.append(changeLastSumPriceBill)
paramList.append(entry)
entry = []

penultSumBoxMill = df_already_scanned.loc[x-2:x-2, ['dfSumBoxMill']].iloc[0,0]
entry.append('penultSumBoxMill')
entry.append(penultSumBoxMill)
paramList.append(entry)
entry = []

lastSumBoxMill = df_already_scanned.loc[x-1:x, ['dfSumBoxMill']].iloc[0,0]
entry.append('lastSumBoxMill')
entry.append(lastSumBoxMill)
paramList.append(entry)
entry = []

changeLastSumBoxMill = ((lastSumBoxMill/penultSumBoxMill)-1)*100
entry.append('changeLastSumBoxMill')
entry.append(changeLastSumBoxMill)
paramList.append(entry)
entry = []

yesterSumBoxMill = df_already_scanned.loc[x-13:x-12, ['dfSumBoxMill']].iloc[0,0]
entry.append('yesterSumBoxMill')
entry.append(yesterSumBoxMill)
paramList.append(entry)
entry = []

changeYesterSumBoxMill = ((lastSumBoxMill/yesterSumBoxMill)-1)*100
entry.append('changeYesterSumBoxMill')
entry.append(changeYesterSumBoxMill)
paramList.append(entry)
entry = []


yesterSumPriceBill = df_already_scanned.loc[x-13:x-12, ['dfSumPriceBill']].iloc[0,0]
entry.append('yesterSumPriceBill')
entry.append(yesterSumPriceBill)
paramList.append(entry)
entry = []


changeYesterSumPriceBill = ((lastSumPriceBill/yesterSumPriceBill)-1)*100
entry.append('changeYesterSumPriceBill')
entry.append(changeYesterSumPriceBill)
paramList.append(entry)
entry = []

In [7]:
neak_paramList = pd.DataFrame(paramList, columns =['parameter', 'value'])

In [8]:
neak_paramList

,parameter,value
0,sumRecord_Number,2.400458e+07
1,minSumPriceBill,3.180627e+01
2,maxSumPriceBill,5.917420e+01
3,avgSumPriceBill,4.001119e+01
4,penultSumPriceBill,4.753280e+01
5,lastSumPriceBill,5.128301e+01
6,changeLastSumPriceBill,7.889737e+00
7,penultSumBoxMill,1.497459e+01
8,lastSumBoxMill,1.582185e+01
9,changeLastSumBoxMill,5.657993e+00


In [7]:
neak_paramList.to_csv("neak_paramList.csv", sep=";", encoding='utf-8', index=False)

## Havi felösszegzések előállítása recent7 -> df_recent7_monthly

In [2]:
# Abbreviations of the Names of the Months (Yale University Library)
# https://web.library.yale.edu/cataloging/months
# line = "English"

monthDict = {
'01':"Jan.",
'02':"Feb.",
'03':"Mar.",
'04':"Apr.",
'05':"May",
'06':"June",
'07':"July",
'08':"Aug.",
'09':"Sept.",
'10':"Oct.",
'11':"Nov.",
'12':"Dec."}

In [4]:
db = sqlite3.connect('neak2.sqlite3')

In [5]:
q = "select idoszak, sum(fogyar) as sumfogyar, sum(doboz) as sumdoboz from recent7 group by idoszak;"

In [6]:
df = pd.read_sql(q, db)

In [8]:
df['year']= df['idoszak'].str.slice(0, 4)

In [9]:
df['honap']= df['idoszak'].str.slice(4, 6)

In [10]:
df['month'] = df['honap'].map(monthDict)

In [23]:
# df.to_csv("recent6_monthly.csv", sep=";", encoding='utf-8', index=False)

In [13]:
df.to_sql(("df_recent7_monthly"), db, if_exists = 'replace')

In [14]:
db.close()

## Havi felösszegzések előállítása whole2022 -> df_atc1_recent

In [27]:
db = sqlite3.connect('neak2.sqlite3')

In [28]:
q = "SELECT idoszak, substr(atc, 1, 1) AS atc1, sum(doboz) AS sumdob, sum(fogyar) AS sumfogyar FROM whole2022 GROUP BY substr(atc, 1, 1), idoszak;"

In [29]:
df = pd.read_sql(q, db)

In [30]:
df

,idoszak,atc1,sumdob,sumfogyar
0,202202,-,5.150000,9.021890e+06
1,202204,-,1.000000,1.804349e+06
2,202205,-,5.000000,3.608698e+06
3,202206,-,8.750000,3.615644e+06
4,202207,-,3.000000,3.609538e+06
...,...,...,...,...
121,202204,V,409392.064689,1.631803e+09
122,202205,V,416376.177128,1.674671e+09
123,202206,V,421033.719723,1.702956e+09
124,202207,V,404909.643424,1.649154e+09


In [33]:
df_atc1 = df.pivot(index='atc1', columns='idoszak', values='sumdob').fillna(0)

In [34]:
df_atc1ft = df.pivot(index='atc1', columns='idoszak', values='sumfogyar').fillna(0)

In [35]:
df_atc1

idoszak,202201,202202,202203,202204,202205,202206,202207,202208
atc1,,,,,,,,
-,0.000000e+00,5.150000e+00,0.000000e+00,1.000000e+00,5.000000e+00,8.750000e+00,3.000000e+00,1.000000e+00
A,2.822996e+06,2.854347e+06,3.076204e+06,2.740975e+06,2.831275e+06,2.825518e+06,2.673257e+06,2.836903e+06
B,1.027582e+06,1.013524e+06,1.115631e+06,1.012483e+06,1.069163e+06,1.075053e+06,1.013221e+06,1.095575e+06
C,5.590668e+06,5.540089e+06,6.179563e+06,5.573265e+06,5.912467e+06,5.938523e+06,5.542669e+06,5.846233e+06
D,2.458376e+05,2.452091e+05,2.905054e+05,2.495954e+05,2.691559e+05,2.581992e+05,2.354804e+05,2.360074e+05
G,2.318122e+05,2.285869e+05,2.576791e+05,2.325525e+05,2.472390e+05,2.474284e+05,2.245441e+05,2.377548e+05
H,2.008414e+05,1.989373e+05,2.283020e+05,2.016805e+05,2.118293e+05,2.090617e+05,1.913742e+05,1.992098e+05
I,1.436083e+06,1.305168e+06,1.512923e+06,1.339834e+06,1.388955e+06,1.261753e+06,1.121245e+06,1.187223e+06
J,6.574399e+05,6.568671e+05,7.590046e+05,5.926986e+05,5.430780e+05,4.735648e+05,4.586289e+05,4.821281e+05


In [36]:
df_atc1ft

idoszak,202201,202202,202203,202204,202205,202206,202207,202208
atc1,,,,,,,,
-,0.000000e+00,9.021890e+06,0.000000e+00,1.804349e+06,3.608698e+06,3.615644e+06,3.609538e+06,1.804349e+06
A,9.070201e+09,9.237626e+09,1.023046e+10,9.223284e+09,9.729720e+09,9.905000e+09,9.465499e+09,9.995631e+09
B,5.585106e+09,5.486753e+09,6.142151e+09,5.672033e+09,6.103235e+09,6.147394e+09,5.815425e+09,6.322889e+09
C,7.941015e+09,7.884912e+09,8.819301e+09,8.013763e+09,8.518707e+09,8.542958e+09,7.989127e+09,8.513935e+09
D,4.504471e+08,4.491802e+08,5.179559e+08,4.741243e+08,5.162438e+08,5.143592e+08,4.658089e+08,4.838408e+08
G,1.118440e+09,1.099868e+09,1.218036e+09,1.136669e+09,1.224166e+09,1.189387e+09,9.583523e+08,1.066668e+09
H,9.049444e+08,8.948981e+08,9.979394e+08,9.105553e+08,9.470900e+08,9.315195e+08,8.715443e+08,8.969208e+08
I,7.217619e+08,6.438635e+08,7.438103e+08,7.239512e+08,7.517459e+08,6.960119e+08,6.450958e+08,6.600117e+08
J,2.013880e+09,1.953068e+09,2.296347e+09,1.873182e+09,1.844895e+09,1.729218e+09,1.782991e+09,2.290598e+09


In [31]:
# df.to_csv("neak_atc1.csv", sep=";", encoding='utf-8', index=False)

In [37]:
df_atc1.to_sql(("df_atc1_2022_dob"), db, if_exists = 'replace')

In [38]:
df_atc1ft.to_sql(("df_atc1_2022_ft"), db, if_exists = 'replace')

In [39]:
db.close()

## Havi felösszegzések előállítása whole2022 -> df_atc_idoszak

In [53]:
db = sqlite3.connect('neak2.sqlite3')

In [54]:
q = "SELECT idoszak, atc, sum(doboz) AS sumdob, sum(fogyar) AS sumfogyar FROM whole2022 GROUP BY idoszak, atc;"

In [60]:
df = pd.read_sql(q, db)

In [61]:
df['atc1']= df['atc'].str.slice(0, 1)

In [62]:
df['atc3']= df['atc'].str.slice(0, 3)

In [63]:
df['atc5']= df['atc'].str.slice(0, 5)

In [ ]:
df['year']= df['idoszak'].str.slice(0, 4)

In [ ]:
df

In [65]:
df.to_sql(("df_atc_idoszak"), db, if_exists = 'replace')

In [66]:
db.close()

## Havi felösszegzések előállítása whole2022 -> df_megye22

In [67]:
db = sqlite3.connect('neak2.sqlite3')

In [68]:
q = "SELECT idoszak, megye, sum(doboz) AS sumdob, sum(fogyar) AS sumfogyar FROM whole2022 GROUP BY idoszak, megye;"

In [69]:
df = pd.read_sql(q, db)

In [70]:
# https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjki_OS-5v5AhVN2aQKHajIDVkQFnoECBEQAQ&url=https%3A%2F%2Fwww.ksh.hu%2Fdocs%2Fhun%2Fxftp%2Froviditesek.pdf&usg=AOvVaw2_-SdIQNO8KCskUhnIm9gd
# KSH módszertani útmutató
# Megyék rövidítésjegyzéke

megyedict = {"BA":"Baranya",
"BO":"Borsod-Abaúj-Zemplén",
"BP":"Budapest",
"BÁ":"Bács-Kiskun",
"BÉ":"Békés",
"CS":"Csongrád-Csanád",
"FE":"Fejér",
"GY":"Győr-Moson-Sopron",
"HA":"Hajdú-Bihar",
"HE":"Heves",
"JÁ":"Jász-Nagykun-Szolnok",
"KO":"Komárom-Esztergom",
"NÓ":"Nógrád",
"PE":"Pest",
"SO":"Somogy",
"SZ":"Szabolcs-Szatmár-Bereg",
"TO":"Tolna",
"VA":"Vas",
"VE":"Veszprém",
"ZA":"Zala"}

In [71]:
df['megyekod'] = df['megye']

In [72]:
df['megye'] = df['megye'].map(megyedict)

In [73]:
df.sort_values(by=['megye','idoszak'], inplace=True)

In [75]:
df_megye = df.pivot(index='megye', columns='idoszak', values='sumfogyar').fillna(0)

In [ ]:
df_megye

In [77]:
#df.to_csv("neak_megye.csv", sep=";", encoding='utf-8', index=False)
df_megye.to_sql(("df_megye22"), db, if_exists = 'replace')

In [78]:
db.close()

## Éves összegzések letöltése SQLite -> .csv (DataWrapper)

In [2]:
db = sqlite3.connect('neak2.sqlite3')

In [3]:
q = "select * from years"
df_years = pd.read_sql(q, db)

### kialakítok egy éves teljes táblát, amelyből származtatok több speciális tablót a DW részére

In [4]:
sumdob2022 = df_years.iloc[16,1]/8*12
sumtb2022 = df_years.iloc[16,2]/8*12
sumfogyar2022 = df_years.iloc[16,3]/8*12
sumterdij2022 = df_years.iloc[16,4]/8*12

In [5]:
df_years.iloc[16,0] = 'CALC2022'
df_years.iloc[16,1] = sumdob2022
df_years.iloc[16,2] = sumtb2022
df_years.iloc[16,3] = sumfogyar2022
df_years.iloc[16,4] = sumterdij2022

In [6]:
df_years['allami'] = df_years['sumfogyar'] - df_years['sumterdij']

In [ ]:
# ez a félkész alaptábla
df_years

In [9]:
# mivel csak egy kategóriával dolgozunk az éves változásokhoz használhatjuk a .shift() funkciót
# így idő-intelligenciát használunk

df_years['changeSumDob'] = (((df_years['sumdob']/df_years.shift(1, axis = 0)['sumdob'])-1)*100).round(decimals=2)
df_years['changeAllami'] = (((df_years['allami']/df_years.shift(1, axis = 0)['allami'])-1)*100).round(decimals=2)
df_years['changeSumTerdij'] = (((df_years['sumterdij']/df_years.shift(1, axis = 0)['sumterdij'])-1)*100).round(decimals=2)
df_years = df_years.fillna(0)

In [ ]:
# ez a kész alaptábla

df_years

In [12]:
# df_years1: Vényes gyógyszerforgalom és az állam költsége

df_years1 = df_years.drop(columns=['sumdob', 'sumtb', 'sumfogyar', 'ttt_db', 'changeSumDob', 'changeAllami', 'changeSumTerdij'])

In [ ]:
df_years1

In [14]:
# Éves változások százalékban 2006-tól

df_years2 = df_years.drop(columns=['sumdob', 'sumtb', 'sumfogyar', 'sumterdij', 'ttt_db', 'allami', 'changeSumDob'])

In [ ]:
df_years2

In [21]:
# df_years3: Vényes dobozforgalom alakulása

df_years3 = df_years.drop(columns=['sumdob','sumtb', 'sumfogyar', 'sumterdij', 'ttt_db', 'allami', 'changeAllami', 'changeSumTerdij'])

In [ ]:
df_years3

In [16]:
df_years1.to_sql(("df_years1"), db, if_exists = 'replace')
df_years2.to_sql(("df_years2"), db, if_exists = 'replace')

In [17]:
df_years1.to_csv("neak_years1.csv", sep=";", encoding='utf-8', index=False)

In [18]:
df_years2.to_csv("neak_years2.csv", sep=";", encoding='utf-8', index=False)

In [23]:
df_years3.to_sql(("df_years3"), db, if_exists = 'replace')

In [24]:
df_years3.to_csv("neak_years3.csv", sep=";", encoding='utf-8', index=False)

### DataWrapper: éves bontású ATC1 tábla leszedése

In [2]:
db = sqlite3.connect('neak2.sqlite3')

In [3]:
# ezt minden hónapban egyszer le kell futtatni

q = "select * from atc1"   # ezt minden hónapban egyszer le kell futtatni
df_atc1 = pd.read_sql(q, db)    # ezt minden hónapban egyszer le kell futtatni

In [5]:
df_atc1.to_sql(("df_atc1"), db, if_exists = 'replace')   # ezt minden hónapban egyszer le kell futtatni

In [9]:
q = "select * from df_atc1"
df_atc1 = pd.read_sql(q, db)

In [11]:
# éves bontású ATC1 fogyár forgalommal

df_atc1FtChange = df_atc1.pivot(index='atc1', columns='year', values='sumfogyar').fillna(0)

In [12]:
df_atc1FtChange = df_atc1FtChange[1:]   # első sor kihagyása

In [13]:
df_atc1FtChange = df_atc1FtChange.head(15)  # utolsó sor kihagyása

In [15]:
df_atc1FtChange = df_atc1FtChange[['2006', 'v2022MAT']]   #csak két oszlop kiválasztása

In [ ]:
df_atc1FtChange

In [17]:
# éves bontású ATC1 doboz forgalommal

df_atc1DbChange = df_atc1.pivot(index='atc1', columns='year', values='sumdob').fillna(0)

In [ ]:
df_atc1DbChange

In [ ]:
df_atc1DbChange = df_atc1DbChange[1:]   # első sor kihagyása
df_atc1DbChange = df_atc1DbChange.head(15)  # utolsó sor kihagyása
df_atc1DbChange = df_atc1DbChange[['2006', 'v2022MAT']]   #csak két oszlop kiválasztása
df_atc1DbChange

In [20]:
df_atc1FtChange.to_sql(("df_atc1FtChange"), db, if_exists = 'replace')
df_atc1DbChange.to_sql(("df_atc1DbChange"), db, if_exists = 'replace')

In [ ]:
df_atc1DbChange

In [22]:
df_atc1FtChange.to_csv("neak_atc1FtChange.csv", sep=";", encoding='utf-8', index=True)
df_atc1DbChange.to_csv("neak_atc1DbChange.csv", sep=";", encoding='utf-8', index=True)

In [23]:
db.close()

### DataWrapper: éves bontású ATC tábla: view atc_idoszak -> DataWrapper felé .csv

In [1]:
atcDict = {"A01":"stomatological preparations",
"A02":"antacids, drugs for treatment of peptic ulcer and flatulence",
"A03":"antispasmodic and anticholinergic agents and propulsives",
"A04":"antiemetics and antinauseants",
"A05":"bile and liver therapy",
"A06":"laxatives",
"A07":"antidiarrheals, intestinal antiinflammatory/antiinfective agents",
"A08":"antiobesity preparations, excl. diet products",
"A09":"digestives, incl. enzymes",
"A10":"drugs used in diabetes",
"A11":"vitamins",
"A12":"supplements",
"A13":"tonics",
"A14":"anabolic agents for systemic use",
"A15":"appetite stimulants",
"A16":"other alimentary tract and metabolism products",
"B01":"antithrombotic agents",
"B02":"antihemorrhagics",
"B03":"antianemic preparations",
"B04":"lipid modifying agents",
"B05":"blood substitutes and perfusion solutions",
"B06":"other hematological agents",
"C01":"cardiac therapy",
"C02":"antihypertensives",
"C03":"diuretics",
"C04":"peripheral vasodilators",
"C05":"vasoprotectives",
"C07":"beta blocking agents",
"C08":"calcium channel blockers",
"C09":"agents acting on the renin-angiotensin system",
"C10":"serum lipid reducing agents",
"D01":"antifungals for dermatological use",
"D02":"emollients and protectives",
"D03":"preparations for treatment of wounds and ulcers",
"D04":"antipruritics, incl. antihistamines, anesthetics, etc.",
"D05":"antipsoriatics",
"D06":"antibiotics and chemotherapeutics for dermatological use",
"D07":"corticosteroids, dermatological preparations",
"D08":"antiseptics and disinfectants",
"D09":"medicated dressings",
"D10":"anti-acne preparations",
"D11":"other dermatological preparations",
"G01":"gynecological antiinfectives and antiseptics",
"G02":"other gynecologicals",
"G03":"sex hormones and modulators of the genital system",
"G04":"urologicals",
"H01":"pituitary and hypothalamic hormones and analogues",
"H02":"corticosteroids for systemic use",
"H03":"thyroid therapy",
"H04":"pancreatic hormones",
"H05":"calcium homeostasis",
"H10":"other hormones",
"ISM":"unknown",
"J01":"antibacterials for systemic use",
"J02":"antimycotics for systemic use",
"J03":"other J",
"J04":"antimycobacterials",
"J05":"antivirals for systemic use",
"J06":"immune sera and immunoglobulins",
"J07":"vaccines",
"J08":"other antiinfectives, unknown",
"L01":"antineoplastic agents",
"L02":"endocrine therapy",
"L03":"immunostimulants",
"L04":"immunosuppressive agents",
"M01":"antiinflammatory and antirheumatic products",
"M02":"topical products for joint and muscular pain",
"M03":"muscle relaxants",
"M04":"antigout preparations",
"M05":"drugs for treatment of bone diseases",
"M09":"other drugs for disorders of the musculo-skeletal system",
"N01":"anesthetics",
"N02":"analgesics",
"N03":"antiepileptics",
"N04":"anti-parkinson drugs",
"N05":"psycholeptics",
"N06":"psychoanaleptics",
"N07":"other nervous system drugs",
"NO7":"unknown N",
"P01":"antiprotozoals",
"P02":"anthelmintics",
"P03":"ectoparasiticides, incl. scabicides, insecticides and repellents",
"R01":"nasal preparations",
"R02":"throat preparations",
"R03":"anti-asthmatics",
"R04":"other R",
"R05":"cough and cold preparations",
"R06":"antihistamines for systemic use",
"R07":"other respiratory system products",
"S01":"ophthalmologicals",
"S02":"otologicals",
"S03":"ophthalmological and otological preparations",
"V01":"allergens",
"V02":"immunsupressiva",
"V03":"all other therapeutic products",
"V04":"diagnostic agents",
"V05":"surgical antiseptics",
"V06":"general nutrients",
"V07":"all other non-therapeutic products",
"V08":"contrast media",
"V09":"diagnostic radiopharmaceuticals",
"V10":"therapeutic radiopharmaceuticals",
"V20":"surgical dressings"}

#### ATC = "J01" csoportba tartozó hatóanyagok

In [15]:
db = sqlite3.connect('neak2.sqlite3')

In [54]:
q = "select * from atc_idoszak where atc like 'J01%';"
df = pd.read_sql(q, db)

In [55]:
df['year']= df['idoszak'].str.slice(0, 4)

In [56]:
df['atc5'] = df['atc'].str.slice(0, 5)

In [57]:
atcJDict = {'J01AA': 'J01REST',
'J01C ': 'J01REST',
'J01CA': 'J01REST',
'J01CE': 'J01REST',
'J01CF': 'J01REST',
'J01CR': 'J01CR',
'J01DA': 'J01REST',
'J01DB': 'J01REST',
'J01DC': 'J01DC',
'J01DD': 'J01DD',
'J01DE': 'J01REST',
'J01DH': 'J01REST',
'J01EE': 'J01REST',
'J01FA': 'J01FA',
'J01FF': 'J01FF',
'J01GA': 'J01REST',
'J01GB': 'J01REST',
'J01MA': 'J01MA',
'J01MB': 'J01REST',
'J01XA': 'J01REST',
'J01XB': 'J01REST',
'J01XD': 'J01REST',
'J01XE': 'J01REST',
'J01XX': 'J01REST'}

In [58]:
df['atc5'] = df['atc5'].map(atcJDict)

In [64]:
dfp = df.pivot_table(index=['atc5'], columns='year', values='sumfogyar', aggfunc= 'sum', margins = True, margins_name='Total').fillna(0)

In [66]:
dfp.reset_index(inplace=True)
dfp = dfp.rename(columns = {'index':'atc5'})
dfp = dfp.drop(columns=['Total'])
# dfp['atcmasik'] = dfp['atc']

In [ ]:
dfp

In [68]:
dfp.to_csv("neak_atc5JYearly.csv", sep=";", encoding='utf-8', index=False)

In [69]:
db.close()

#### ATC = "L" csoportba tartozó hatóanyagcsoportok ATC3 szinten

In [4]:
db = sqlite3.connect('neak2.sqlite3')

In [5]:
q = "select * from atc3 where atc3 like 'L%';"
df = pd.read_sql(q, db)

In [6]:
df = df.pivot_table(index=['atc3'], columns='year', values='sumfogyar', aggfunc= 'sum', margins = True, margins_name='Total').fillna(0)

In [8]:
df = df.drop(columns=['2022', 'Total'])

In [10]:
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'atc3'})

In [11]:
df['atcDescription'] = df['atc3']
df['atcDescription'] = df['atcDescription'].map(atcDict)

In [13]:
df.to_csv("neak_atc3LYearly.csv", sep=";", encoding='utf-8', index=False)

In [14]:
db.close()

### DataWrapper: éves bontású ATC3 tábla: view atc3 -> df_atc3

In [ ]:
db = sqlite3.connect('neak2.sqlite3')

#### ATC3 = "V"

In [49]:
q = "select * from atc3 where atc3 like 'V%';"
df = pd.read_sql(q, db)

In [52]:
# df = df.pivot(index=['atc1', 'atc3'], columns='year', values='sumfogyar').fillna(0)

df = df.pivot_table(index=['atc3'], columns='year', values='sumfogyar', aggfunc= 'sum', margins = True, margins_name='Total').fillna(0)

In [55]:
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'atc3'})

In [57]:
df = df.drop(columns=['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'Total'])

In [58]:
df['change'] = ((df['v2022MAT']/df['2006'])-1)*100

In [60]:
df['atcDescription'] = df['atc3']

In [61]:
df['atcDescription'] = df['atcDescription'].map(atcDict)

In [63]:
df.fillna('Total')

year,atc3,2006,v2022MAT,change,atcDescription
0,V01,1.188866e+07,3.645538e+07,206.640009,allergens
1,V03,6.873529e+08,6.459800e+08,-6.019169,all other therapeutic products
2,V04,2.551811e+07,4.496552e+07,76.210225,diagnostic agents
3,V06,7.606720e+09,1.915908e+10,151.870402,general nutrients
4,V07,1.070456e+08,4.050798e+05,-99.621582,all other non-therapeutic products
5,V08,2.994750e+05,0.000000e+00,-100.000000,contrast media
6,V10,5.610173e+07,0.000000e+00,-100.000000,therapeutic radiopharmaceuticals
7,Total,8.494926e+09,1.988688e+10,134.103051,Total


In [64]:
df.to_csv("neak_atc3V.csv", sep=";", encoding='utf-8', index=True)

### ATC3 = "I"

In [65]:
q = "select * from atc3 where atc3 like 'I%';"
df = pd.read_sql(q, db)

In [67]:
df = df.pivot_table(index=['atc3'], columns='year', values='sumfogyar', aggfunc= 'sum', margins = True, margins_name='Total').fillna(0)

In [68]:
df

year,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,v2022MAT,Total
atc3,,,,,,,,,,,,,,,,,,,
ISM,5.331336e+08,2.792590e+08,1.203226e+08,6.548920e+09,6.681486e+09,6.722886e+09,6.859036e+09,7.311039e+09,7.855937e+09,8.325405e+09,8.555458e+09,8.697366e+09,8.796175e+09,8.787115e+09,8.029687e+09,8.417309e+09,5.586252e+09,8.572221e+09,1.166790e+11
Total,5.331336e+08,2.792590e+08,1.203226e+08,6.548920e+09,6.681486e+09,6.722886e+09,6.859036e+09,7.311039e+09,7.855937e+09,8.325405e+09,8.555458e+09,8.697366e+09,8.796175e+09,8.787115e+09,8.029687e+09,8.417309e+09,5.586252e+09,8.572221e+09,1.166790e+11


#### ATC3 = "L"

In [69]:
q = "select * from atc3 where atc3 like 'L%';"
df = pd.read_sql(q, db)

In [70]:
df = df.pivot_table(index=['atc3'], columns='year', values='sumfogyar', aggfunc= 'sum', margins = True, margins_name='Total').fillna(0)

In [72]:
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'atc3'})

In [74]:
df = df.drop(columns=['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'Total'])

In [76]:
df['change'] = ((df['v2022MAT']/df['2006'])-1)*100
df['atcDescription'] = df['atc3']
df['atcDescription'] = df['atcDescription'].map(atcDict)

year,atc3,2006,v2022MAT,change,atcDescription
0,L01,7.170716e+09,6.251610e+10,771.825094,antineoplastic agents
1,L02,1.483304e+10,9.850184e+09,-33.592935,endocrine therapy
2,L03,7.480114e+09,6.791969e+09,-9.199652,immunostimulants
3,L04,6.522477e+09,2.754507e+10,322.309937,immunosuppressive agents
4,Total,3.600634e+10,1.067033e+11,196.345907,Total


In [81]:
df.to_csv("neak_atc3L.csv", sep=";", encoding='utf-8', index=False)

#### ATC3 = "B"

In [82]:
q = "select * from atc3 where atc3 like 'B%';"
df = pd.read_sql(q, db)

In [84]:
df = df.pivot_table(index=['atc3'], columns='year', values='sumfogyar', aggfunc= 'sum', margins = True, margins_name='Total').fillna(0)

In [85]:
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'atc3'})
df = df.drop(columns=['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'Total'])
df['change'] = ((df['v2022MAT']/df['2006'])-1)*100
df['atcDescription'] = df['atc3']
df['atcDescription'] = df['atcDescription'].map(atcDict)

In [88]:
df.to_csv("neak_atc3B.csv", sep=";", encoding='utf-8', index=False)

#### ATC3 = "M"

In [93]:
q = "select * from atc3 where atc3 like 'M%';"
df = pd.read_sql(q, db)

In [94]:
df = df.pivot_table(index=['atc3'], columns='year', values='sumfogyar', aggfunc= 'sum', margins = True, margins_name='Total').fillna(0)

In [95]:
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'atc3'})
df = df.drop(columns=['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'Total'])
df['change'] = ((df['v2022MAT']/df['2006'])-1)*100
df['atcDescription'] = df['atc3']
df['atcDescription'] = df['atcDescription'].map(atcDict)

In [96]:
df.to_csv("neak_atc3M.csv", sep=";", encoding='utf-8', index=False)

#### ATC3 = "D"

In [97]:
q = "select * from atc3 where atc3 like 'D%';"
df = pd.read_sql(q, db)

In [98]:
df = df.pivot_table(index=['atc3'], columns='year', values='sumfogyar', aggfunc= 'sum', margins = True, margins_name='Total').fillna(0)

In [99]:
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'atc3'})
df = df.drop(columns=['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'Total'])
df['change'] = ((df['v2022MAT']/df['2006'])-1)*100
df['atcDescription'] = df['atc3']
df['atcDescription'] = df['atcDescription'].map(atcDict)

In [100]:
df.to_csv("neak_atc3D.csv", sep=";", encoding='utf-8', index=False)

#### ATC3 = "C"

In [101]:
q = "select * from atc3 where atc3 like 'C%';"
df = pd.read_sql(q, db)

In [102]:
df = df.pivot_table(index=['atc3'], columns='year', values='sumfogyar', aggfunc= 'sum', margins = True, margins_name='Total').fillna(0)

In [103]:
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'atc3'})
df = df.drop(columns=['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'Total'])
df['change'] = ((df['v2022MAT']/df['2006'])-1)*100
df['atcDescription'] = df['atc3']
df['atcDescription'] = df['atcDescription'].map(atcDict)

In [104]:
df.to_csv("neak_atc3C.csv", sep=";", encoding='utf-8', index=False)

#### ATC3 = "J"

In [105]:
q = "select * from atc3 where atc3 like 'J%';"
df = pd.read_sql(q, db)

In [106]:
df = df.pivot_table(index=['atc3'], columns='year', values='sumfogyar', aggfunc= 'sum', margins = True, margins_name='Total').fillna(0)

In [107]:
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'atc3'})
df = df.drop(columns=['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'Total'])
df['change'] = ((df['v2022MAT']/df['2006'])-1)*100
df['atcDescription'] = df['atc3']
df['atcDescription'] = df['atcDescription'].map(atcDict)

In [109]:
df.to_csv("neak_atc3J.csv", sep=";", encoding='utf-8', index=False)

In [110]:
db.close()

### DataWrapper: éves bontású megyei tábla leszedése

In [2]:
db = sqlite3.connect('neak2.sqlite3')
q = "select * from megye20062022;"      # minden hónapban egyszer kell lefuttatni
df = pd.read_sql(q, db)         # minden hónapban egyszer kell lefuttatni

In [ ]:
df

In [5]:
# https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjki_OS-5v5AhVN2aQKHajIDVkQFnoECBEQAQ&url=https%3A%2F%2Fwww.ksh.hu%2Fdocs%2Fhun%2Fxftp%2Froviditesek.pdf&usg=AOvVaw2_-SdIQNO8KCskUhnIm9gd
# KSH módszertani útmutató
# Megyék rövidítésjegyzéke

megyedict = {"BA":"Baranya",
"BO":"Borsod-Abaúj-Zemplén",
"BP":"Budapest",
"BÁ":"Bács-Kiskun",
"BÉ":"Békés",
"CS":"Csongrád-Csanád",
"FE":"Fejér",
"GY":"Győr-Moson-Sopron",
"HA":"Hajdú-Bihar",
"HE":"Heves",
"JÁ":"Jász-Nagykun-Szolnok",
"KO":"Komárom-Esztergom",
"NÓ":"Nógrád",
"PE":"Pest",
"SO":"Somogy",
"SZ":"Szabolcs-Szatmár-Bereg",
"TO":"Tolna",
"VA":"Vas",
"VE":"Veszprém",
"ZA":"Zala"}

In [6]:
df['megyekod'] = df['megye']
df['megye'] = df['megyekod'].map(megyedict)

In [9]:
df_megyeFT = df.pivot(index='megye', columns='year', values='sumfogyar').fillna(0)

In [21]:
df_megyeFT['change'] = (((df_megyeFT['v2022MAT'] / df_megyeFT['2006'])-1)*100).round(decimals=2)

In [ ]:
df_megyeFT

In [15]:
df_megyeFT.to_csv("neak_megyeFT.csv", sep=";", encoding='utf-8', index=True)

In [17]:
df_megyedb = df.pivot(index='megye', columns='year', values='sumdob').fillna(0)

In [23]:
df_megyedb['change'] = (((df_megyedb['v2022MAT'] / df_megyedb['2006'])-1)*100).round(decimals=2)

In [ ]:
df_megyedb

In [25]:
df_megyedb.to_csv("neak_megyedb.csv", sep=";", encoding='utf-8', index=True)

In [26]:
db.close()